<a href="https://colab.research.google.com/github/rayhannarindran/fitmotion_model/blob/main/trainingScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

# **Dataset Initiation**

In [15]:
# change these following three lines only
subject_data_file = 'data_subjects_info.csv'
data_dir = 'dataset'

# List all files in a directory using os.listdir
def get_file_paths(data_dir):
    files = []
    for dirname, _, filenames in os.walk(data_dir):
        for filename in filenames:
            files.append(os.path.join(dirname, filename))
    return files

def get_all_data(file_paths, meta):
    df = pd.DataFrame()

    for p in file_paths:
        print(p)
        c_dir, c_file = p.split('\\')[-2], p.split('\\')[-1]
        c_cat, c_ses = c_dir.split('_')[-2], c_dir.split('_')[-1]
        c_sub = c_file.split('_')[-1].split('.')[-2]
        
        tdf = pd.read_csv(p, encoding = "utf-8")
        tdf = tdf.assign(subject_id = int(c_sub))
        tdf = tdf.assign(session_id = int(c_ses))
        tdf = tdf.assign(category = str(c_cat))
        tdf = tdf.assign(age = int(meta.age[int(c_sub) - 1]))
        tdf = tdf.assign(gender = int(meta.gender[int(c_sub) - 1]))
        tdf = tdf.assign(height = int(meta.height[int(c_sub) - 1]))
        tdf = tdf.assign(weight = int(meta.weight[int(c_sub) - 1]))

        df = pd.concat([df, tdf])
    df.reset_index(drop=True, inplace=True)
    return df
    
subject_data_frame = pd.read_csv(subject_data_file)
file_paths = get_file_paths(data_dir)
data_frame = get_all_data(file_paths, subject_data_frame)

dataset\dws_1\sub_1.csv
dataset\dws_1\sub_10.csv
dataset\dws_1\sub_11.csv
dataset\dws_1\sub_12.csv
dataset\dws_1\sub_13.csv
dataset\dws_1\sub_14.csv
dataset\dws_1\sub_15.csv
dataset\dws_1\sub_16.csv
dataset\dws_1\sub_17.csv
dataset\dws_1\sub_18.csv
dataset\dws_1\sub_19.csv
dataset\dws_1\sub_2.csv
dataset\dws_1\sub_20.csv
dataset\dws_1\sub_21.csv
dataset\dws_1\sub_22.csv
dataset\dws_1\sub_23.csv
dataset\dws_1\sub_24.csv
dataset\dws_1\sub_3.csv
dataset\dws_1\sub_4.csv
dataset\dws_1\sub_5.csv
dataset\dws_1\sub_6.csv
dataset\dws_1\sub_7.csv
dataset\dws_1\sub_8.csv
dataset\dws_1\sub_9.csv
dataset\dws_11\sub_1.csv
dataset\dws_11\sub_10.csv
dataset\dws_11\sub_11.csv
dataset\dws_11\sub_12.csv
dataset\dws_11\sub_13.csv
dataset\dws_11\sub_14.csv
dataset\dws_11\sub_15.csv
dataset\dws_11\sub_16.csv
dataset\dws_11\sub_17.csv
dataset\dws_11\sub_18.csv
dataset\dws_11\sub_19.csv
dataset\dws_11\sub_2.csv
dataset\dws_11\sub_20.csv
dataset\dws_11\sub_21.csv
dataset\dws_11\sub_22.csv
dataset\dws_11\sub_23

In [16]:
# Show the first 5 rows of the dataset
data_frame.info()

         Unnamed: 0  attitude.roll  attitude.pitch  attitude.yaw  gravity.x  \
0                 0       1.528132       -0.733896      0.696372   0.741895   
1                 1       1.527992       -0.716987      0.677762   0.753099   
2                 2       1.527765       -0.706999      0.670951   0.759611   
3                 3       1.516768       -0.704678      0.675735   0.760709   
4                 4       1.493941       -0.703918      0.672994   0.760062   
...             ...            ...             ...           ...        ...   
1412860        4749       0.890518       -1.300928      0.089878   0.207249   
1412861        4750       0.929499       -1.319499      0.124063   0.199247   
1412862        4751       0.966103       -1.340284      0.159858   0.187951   
1412863        4752       1.005158       -1.362495      0.201540   0.174576   
1412864        4753       1.051479       -1.385380      0.249462   0.160035   

         gravity.y  gravity.z  rotationRate.x  rota

In [19]:
labels = ['dws', 'ups', 'wlk', 'jog', 'std']

# **Model Training**

# **Model Results and Testing**